In [1]:
import pandas as pd

In [29]:
df = pd.read_csv("E:/!!Research/!!!Data/graph_analysis/lek_data/lek_data_binary_cleaned.csv")
df.head()

,lek_id,year,activity,x_easting,y_northing
0,2,2007,True,672527,3730500
1,2,2008,True,672527,3730500
2,2,2009,True,672527,3730500
3,2,2010,True,672527,3730500
4,4,2007,True,669625,3729786


In [30]:
# Define the range of years you want
years = range(df['year'].min(), df['year'].max() + 1)

# Create a complete DataFrame for all leks and years
leks = df['lek_id'].unique()
complete_index = pd.MultiIndex.from_product([leks, years], names=['lek_id', 'year'])
complete_df = pd.DataFrame(index=complete_index).reset_index()

# Check the columns in the original DataFrame
#print("Original DataFrame columns:", df.columns)

# Create a unique DataFrame to maintain site info and coordinates
unique_sites_df = df[['lek_id', 'x_easting', 'y_northing']].drop_duplicates()

# Merge the unique sites DataFrame with the complete DataFrame
result_df = pd.merge(complete_df, unique_sites_df, on='lek_id', how='left')

# Now merge the activity data
df = pd.merge(result_df, df[['lek_id', 'year', 'activity']], on=['lek_id', 'year'], how='left')

# Replace NaN values in the 'activity' column with "not surveyed"
df['activity'] = df['activity'].fillna('not surveyed')

# Display the result
print(df)

       lek_id  year  x_easting  y_northing      activity
0           2  1971     672527     3730500  not surveyed
1           2  1972     672527     3730500  not surveyed
2           2  1973     672527     3730500  not surveyed
3           2  1974     672527     3730500  not surveyed
4           2  1975     672527     3730500  not surveyed
...       ...   ...        ...         ...           ...
26587   99999  2014     649863     3714293         False
26588   99999  2015     649863     3714293  not surveyed
26589   99999  2016     649863     3714293  not surveyed
26590   99999  2017     649863     3714293  not surveyed
26591   99999  2018     649863     3714293  not surveyed

[26592 rows x 5 columns]


In [31]:
# Convert 'year' column to datetime for time calculations
df['year'] = pd.to_datetime(df['year'], format='%Y')

# Sort by lek and year
df = df.sort_values(by=['lek_id', 'year'])

# Initialize a new column for activity in the last five years
df['active_last_5_years'] = False

# Modified function to check last 5 years and reset the index for proper alignment
def check_activity_last_5_years(group):
    years = group['year'].dt.year
    activity_status = []
    
    for i in range(len(group)):
        # Define the 5-year range
        start_year = years.iloc[i] - 4
        # Slice to get the last 5 years' activities
        recent_activities = group[(years >= start_year) & (years <= years.iloc[i])]['activity']
        
        # Check if 'True' exists in the last 5 years
        if True in recent_activities.values:
            activity_status.append('True')
        #elif 'not surveyed' not in recent_activities.values and recent_activities.size > 0:
        #    activity_status.append('Not Surveyed Over 5 Years')
        else:
            activity_status.append('Inactive or not surveyed')
    
    # Return a properly aligned series with the original index
    return pd.Series(activity_status, index=group.index)

# Apply the function by lek
df['active_last_5_years'] = df.groupby('lek_id', group_keys=False).apply(check_activity_last_5_years)

print(df)

       lek_id       year  x_easting  y_northing      activity  \
0           2 1971-01-01     672527     3730500  not surveyed   
1           2 1972-01-01     672527     3730500  not surveyed   
2           2 1973-01-01     672527     3730500  not surveyed   
3           2 1974-01-01     672527     3730500  not surveyed   
4           2 1975-01-01     672527     3730500  not surveyed   
...       ...        ...        ...         ...           ...   
26587   99999 2014-01-01     649863     3714293         False   
26588   99999 2015-01-01     649863     3714293  not surveyed   
26589   99999 2016-01-01     649863     3714293  not surveyed   
26590   99999 2017-01-01     649863     3714293  not surveyed   
26591   99999 2018-01-01     649863     3714293  not surveyed   

            active_last_5_years  
0      Inactive or not surveyed  
1      Inactive or not surveyed  
2      Inactive or not surveyed  
3      Inactive or not surveyed  
4      Inactive or not surveyed  
...            

C:\Users\Justin\AppData\Local\Temp\ipykernel_12748\1760865202.py:33: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['active_last_5_years'] = df.groupby('lek_id', group_keys=False).apply(check_activity_last_5_years)


In [32]:
df.to_csv("E:/!!Research/!!!Data/graph_analysis/lek_data/lek_data_binary_rolling_activity.csv")

If wanting to examine more parts of the data, use below chunks.

This chunk is looking at years since last activity.

In [24]:
# Convert 'year' column to datetime for time calculations
df['year'] = pd.to_datetime(df['year'], format='%Y')

# Sort by lek and year
df = df.sort_values(by=['lek_id', 'year'])

# Initialize a new column for activity in the last five years and years since last "True" activity
df['active_last_5_years'] = False
df['years_since_last_active'] = None

# Modified function to check last 5 years and reset the index for proper alignment
def check_activity_last_5_years(group):
    years = group['year'].dt.year
    activity_status = []
    years_since_last_active = []
    last_active_year = None  # To track the most recent year with "True" activity

    for i in range(len(group)):
        # Define the 5-year range
        start_year = years.iloc[i] - 4
        # Slice to get the last 5 years' activities
        recent_activities = group[(years >= start_year) & (years <= years.iloc[i])][['activity', 'year']]

        # Check if 'True' exists in the last 5 years
        if True in recent_activities['activity'].values:
            activity_status.append('True')
            last_active_year = recent_activities[recent_activities['activity'] == True]['year'].max().year
            years_since_last_active.append(years.iloc[i] - last_active_year)
        elif 'not surveyed' not in recent_activities['activity'].values and recent_activities.size > 0:
            activity_status.append('Not Surveyed Over 5 Years')
            years_since_last_active.append(None)  # Not surveyed over 5 years
        else:
            activity_status.append('Inactive or not surveyed')
            if last_active_year is not None:
                years_since_last_active.append(years.iloc[i] - last_active_year)
            else:
                years_since_last_active.append(None)

    # Return two series with proper alignment
    return pd.DataFrame({
        'active_last_5_years': activity_status,
        'years_since_last_active': years_since_last_active
    }, index=group.index)

# Apply the function by lek
df[['active_last_5_years', 'years_since_last_active']] = df.groupby('lek_id', group_keys=False).apply(check_activity_last_5_years)

print(df)

       lek_id       year  x_easting  y_northing      activity  \
0           2 1971-01-01     672527     3730500  not surveyed   
1           2 1972-01-01     672527     3730500  not surveyed   
2           2 1973-01-01     672527     3730500  not surveyed   
3           2 1974-01-01     672527     3730500  not surveyed   
4           2 1975-01-01     672527     3730500  not surveyed   
...       ...        ...        ...         ...           ...   
26587   99999 2014-01-01     649863     3714293         False   
26588   99999 2015-01-01     649863     3714293  not surveyed   
26589   99999 2016-01-01     649863     3714293  not surveyed   
26590   99999 2017-01-01     649863     3714293  not surveyed   
26591   99999 2018-01-01     649863     3714293  not surveyed   

            active_last_5_years  years_since_last_active  
0      Inactive or not surveyed                      NaN  
1      Inactive or not surveyed                      NaN  
2      Inactive or not surveyed           

C:\Users\Justin\AppData\Local\Temp\ipykernel_12748\1513837516.py:46: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df[['active_last_5_years', 'years_since_last_active']] = df.groupby('lek_id', group_keys=False).apply(check_activity_last_5_years)
C:\Users\Justin\AppData\Local\Temp\ipykernel_12748\1513837516.py:46: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df[['active_last_5_years', 'years_since_last_active']] = df.groupby('lek_id', group_keys=False